In [3]:
import numpy as np
from typing import List
from collections import defaultdict
from itertools import permutations

### Класс

In [4]:
class AffineFunction:
    def __init__(self, coefficients: List[int], domain: Polyhedron):
        self.coefs = list(coefficients)
        self.domain = domain
    
    def value(self, x: List[int]):
        if x not in self.domain:
            raise ValueError(f'{x} is not in function domain {domain.vertices()}')
        if len(x) + 1 != len(self.coefs):
            raise ValueError(f'Wrong dimension of x: {len(x)}')
        return sum([a * b for a, b in zip(self.coefs, x)]) + self.coefs[-1]
    
    def _coef_str(self, i):
        if coef == 0:
            return ''
        if i + 1 == len(self.coefs):
            x_part = ''
        else:
            x_part = f'x_{i+1}'
        if coef == 1:
            coef_part = ' + '
        elif coef == -1:
            coef_part = ' - '
        elif coef > 0:
            coef_part = f' + {self.coefs[i]}'
        else:
            coef_part = f' - {-self.coefs[i]}'
        return coef_part + x_part
           
    
    def __str__(self):
        coefs_repr = []
        if len(self.coefs) > 1:
            coefs_repr.append(f'{self.coefs[0]}x_1')
            for i in range(1, len(self.coefs) - 1):
                coefs_repr.append(self._coef_str(i))
            coefs_repr.append(str(self.coefs[-1]))
        else:
            coefs_repr.append(str(self.coefs[-1]))
        coefs_repr = "".join(coefs_repr)
        return f'Affine function {coefs_repr} with domain ' \
                f'{[vert.vector() for vert in self.domain.vertices()]}'

In [578]:
f = AffineFunction([1], Polyhedron(vertices=[[-1], [1]]))
print(f)

Affine function 1 with domain [(-1), (1)]


In [579]:
f = AffineFunction([1, 1], Polyhedron(vertices=[[-1], [1]]))
print(f)

Affine function 1x_11 with domain [(-1), (1)]


In [564]:
class TestAffineFunctionStr:
    def test_const_func(self):
        f = AffineFunction([1], Polyhedron(vertices=[[-1], [1]]))
        assert f.__str__() == 'Affine function 1 with domain [(-1), (1)]'
        
    def test_one_arg_func(self):
        f = AffineFunction([1, 1], Polyhedron(vertices=[[-1], [1]]))
        

In [565]:
TestAffineFunctionStr().test_const_func()

In [5]:
class PiecewiseAffineFunction:
    def __init__(self, affine_pieces: AffineFunction):
        # TODO: check regularity
        # TODO: check convexity
        # TODO: проверить, что значения в вершинах многогранников одинаковые на смежных 
        # кусках и целые
        self.affine_pieces = affine_pieces
    
    def value(self, x: List[int]):
        for piece in self.affine_pieces:
            if x in piece.domain:
                return piece.value(x)
        raise ValueError(f'{x} is not in function domain')
    
    # TODO: аргументы похожи по смыслу, но имеют разный тип - поправить
    def transform(self, phi, phi_inverse):
        for j in range(len(self.affine_pieces)):
            res = self.affine_pieces[j].coefs[:-1] * phi_inverse
            res = res.tolist()[0]
            res.append(self.affine_pieces[j].coefs[-1])
            self.affine_pieces[j].coefs = res
            vertices = []
            for vert in self.affine_pieces[j].domain.vertices():
                vertices.append(phi(vert._vector))
            self.affine_pieces[j].domain = Polyhedron(vertices=vertices)
        

$f(x_1, x_2) =
    \begin{cases}
      x_1 + x_2 + 1 & \text{if $x_1 \in [-1, 0], x_2 \in [-1, 1]$}\\
      - x_1 + x_2 + 1 & \text{if $x_1 \in [0, 1], x_2 \in [-1, 1]$}\\
    \end{cases}   $

In [537]:
class CDP:
    def __init__(self, psi_list: List[PiecewiseAffineFunction], base: Polyhedron):
        # Check that sum of psi is non negative on borders of domains 
        # (check polytop vertices of domains laying inside of function scope)
        for psi in psi_list:
            for piece in psi.affine_pieces:
                for vert in piece.domain.vertices():
                    if vert in base.vertices():
                        try:
                            s = sum([ps.value(vert.vector) for ps in psi_list])
                        except ValueError:
                            continue
                        else:
                            if s <= 0:
                                raise ValueError(
                                    f'Not a valid CDP - sum of psi is {s} on {vert}')
        # Check that sum of psi is non negative on base vertices
        for vert in base.vertices():
            try:
                s = sum([ps.value(vert) for ps in psi_list])
            except ValueError:
                raise ValueError('Not a valid CDP: psi is not defined on base')
            else:
                if s < 0:
                    raise ValueError(
                                    f'Not a valid CDP - sum of psi is {s} on {vert}') 
        self.psi_list = psi_list
        self.base = base
        self.n = len(self.base.vertices()[0]._vector)
        self.k = len(self.base.vertices())
        # TODO: is there a better solution for vertices traversal?
        self.base_adjacency_map = defaultdict(set)
        
    def _build_adjacency_map(self):
        for i in range(k):
            for j in range(i + 1, k):
                if self.base.vertices()[i].is_incident(self.base.vertices()[j]):
                    self.base_adjacency_map[i].insert(j)
                    self.base_adjacency_map[j].insert(i)
    
    def transform_base(self, phi: linear_transformation):
#         if len(phi.matrix()) != len(self.base.vertices()[0]._vector):
#             raise ValueError(f'Wrong dimension of phi: {len(phi.matrix())}')
        vertices = []
        for vert in self.base.vertices():
            vertices.append(phi(vert._vector))
        self.base = Polyhedron(vertices=vertices)
        inv = phi.inverse().matrix()
        # TODO: optimize
        inv = np.array([np.array(row) for row in inv])
        for i in range(len(self.psi_list)):
            self.psi_list[i].transform(phi, inv)
    
    def translate(self, alpha_list: List[int]):
        if len(alpha_list) != len(self.psi_list):
            raise ValueError(f'Length of alpha_list is {len(alpha_list)}, '
                             f'should be {len(self.psi_list)}')
        if sum(alpha_list) != 0:
            raise ValueError('Sum of coefficients should be 0')
        for idx, psi in enumerate(self.psi_list):
            for piece in psi.affine_pieces:
                piece.coefs[-1] += alpha_list[idx]
    
    def shear(self, beta_list: List[int], v: List[int]):
        if len(beta_list) != len(self.psi_list):
            raise ValueError(f'Length of beta_list is {len(beta_list)}, '
                             f'should be {len(self.phi_list)}')
        if sum(beta_list) != 0:
            raise ValueError('Sum of coefficients should be 0')
        for idx, psi in enumerate(self.psi_list):
            for piece in psi.affine_pieces:
                for j, coef in enumerate(v):
                    piece.coefs[j] += coef * beta_list[idx]
    
    def _vert_permutation_is_valid(self, perm):
        '''Check that vertices permutation saves incidence
        '''
        for vert, inc_list in self.base_adjacency_map.items():
            for other_vert in inc_list:
                if not perm[other_vert] in self.base_adjacency_map[perm[vert]]:
                    return False
        return True
    
    def _get_transform_matrix(self, points, point_images):
        left = np.matmul(points.T, points)  
        try:
            inverse = np.linalg.inv(left)
        except np.linalg.LinAlgError:
            raise ValueError(f'Base of the CDP should be non-degenerate '
                             f'(dimension {self.n} is provided, but '
                             'the real dimension is lower)')
        A = np.matmul(np.matmul(point_images, points), inverse)
        return A
    
    def _apply_transform_to_psi(self, phi, phi_inverse):
        pass
                    
    def equal(self, other_cdp: CDP):
        # Check that self.base is convertible to other_cdp.base with some phi
        # Check that sum(psi_i) stays the same on whole base (check on vertices)
        if len(self.base.vertices()) != len(other_cdp.base.vertices()):
            print("lengths are different")
            return False
        G = np.array([np.array(vert._vector) for vert in other_cdp.base.vertices()])
        G = G.T
        for perm in permutations([i for i in range(self.k)]):  
            print("perm ", perm)
            if not self._vert_permutation_is_valid(perm):
                print("perm not valid")
                continue
            V = np.array([np.array(self.base.vertices()[i]._vector) for i in perm])
            # A transforms base of one CDP to the base of another
            A = self._get_transform_matrix(V, G)
            print(A)
            try:
                A_inv = np.linalg.inv(A)
            except np.linalg.LinAlgError:
                continue
            print(A_inv)
            A = linear_transformation(matrix(QQ, A))
            psi_list = []
            for p in self.psi_list:
                psi = deepcopy(p)
                psi.transform(A, A_inv)
#                 print("transformed psi ", psi.affine_pieces[0].domain, 
#                       psi.affine_pieces[0].domain)
                psi_list.append(psi)
            
            all_sums_are_equal = True
            for j, i in enumerate(perm):
                print(self.base.vertices()[i], other_cdp.base.vertices()[j])
                sum1 = sum([p.value(self.base.vertices()[i]) for p in psi_list])
                sum2 = sum([p.value(other_cdp.base.vertices()[j]) for p in other_cdp.psi_list])
                print(sum1, sum2)
                if sum1 != sum2:
                    all_sums_are_equal = False
            if all_sums_are_equal:
                return True
        return False

### Пример невалидного CDP

In [319]:
base = Polyhedron(vertices=[[-1], [1]])
# y = x, x \in [-1, 0]
f_11 = AffineFunction((1, 0), Polyhedron(vertices=[[-1], [0]]))
# y = -x, x \in [0, 1]
f_12 = AffineFunction((-1, 0), Polyhedron(vertices=[[0], [1]]))
f_1 = PiecewiseAffineFunction([f_11, f_12])
# y = 1/2x - 1/2, x \in [-1, 1]
f_2 = PiecewiseAffineFunction([AffineFunction((1/2, -1/2), Polyhedron(vertices=[[-1], [1]]))])

In [320]:
cdp = CDP([f_1, f_2], base)

ValueError: Not a valid CDP - sum of psi is negative on A vertex at (-1)

### Пример валидного CDP

In [499]:
base = Polyhedron(vertices=[[-1], [1]])
# y = x + 1, x \in [-1, 0]
f_11 = AffineFunction((1, 1), Polyhedron(vertices=[[-1], [0]]))
# y = -x + 1, x \in [0, 1]
f_12 = AffineFunction((-1, 1), Polyhedron(vertices=[[0], [1]]))
f_1 = PiecewiseAffineFunction([f_11, f_12])
# y = 1/2x + 1/2, x \in [-1, 1]
f_2 = PiecewiseAffineFunction([AffineFunction((1/2, 1/2), Polyhedron(vertices=[[-1], [1]]))])

In [500]:
cdp = CDP([f_1, f_2], base)

In [501]:
cdp.psi_list[0].affine_pieces[0].coefs

[1, 1]

### Примеры трансформаций

In [502]:
cdp.shear([-1, 1], [-1])
print(cdp.psi_list[0].affine_pieces[0].coefs)
print(cdp.psi_list[1].affine_pieces[0].coefs)

[2, 1]
[-1/2, 1/2]


In [503]:
cdp.translate([1, -1])
print(cdp.psi_list[0].affine_pieces[0].coefs)
print(cdp.psi_list[1].affine_pieces[0].coefs)

[2, 2]
[-1/2, -1/2]


In [504]:
A = matrix(ZZ, [[-1]])
phi = linear_transformation(A)
cdp.transform_base(phi)

<class 'sage.modules.vector_space_morphism.VectorSpaceMorphism'>
<class 'numpy.ndarray'>
<class 'sage.modules.vector_space_morphism.VectorSpaceMorphism'>
<class 'numpy.ndarray'>


In [497]:
print(cdp.psi_list[0].affine_pieces[0].coefs)
print(cdp.psi_list[0].affine_pieces[0].domain.vertices())
print(cdp.psi_list[1].affine_pieces[0].coefs)
print(cdp.psi_list[1].affine_pieces[0].domain.vertices())


[-2, 2]
(A vertex at (0), A vertex at (1))
[0.5, -1/2]
(A vertex at (-1), A vertex at (1))


### Эквивалентность CDP

In [538]:
base1 = Polyhedron(vertices=[[-1], [1]])
# y = x + 1, x \in [-1, 0]
f_11 = AffineFunction((1, 1), Polyhedron(vertices=[[-1], [0]]))
# y = -x + 1, x \in [0, 1]
f_12 = AffineFunction((-1, 1), Polyhedron(vertices=[[0], [1]]))
f_1 = PiecewiseAffineFunction([f_11, f_12])
# y = 1/2x + 1/2, x \in [-1, 1]
f_2 = PiecewiseAffineFunction([AffineFunction((1/2, 1/2), Polyhedron(vertices=[[-1], [1]]))])
cdp1 = CDP([f_1, f_2], base1)

In [539]:
base2 = Polyhedron(vertices=[[-1], [1]])
# y = 2, x \in [-1, 0]
y_11 = AffineFunction((0, 2), Polyhedron(vertices=[[-1], [0]]))
# y = -2x + 2, x \in [0, 1]
y_12 = AffineFunction((-2, 2), Polyhedron(vertices=[[0], [1]]))
y_1 = PiecewiseAffineFunction([y_11, y_12])
# y = 1/2x - 1/2, x \in [-1, 1]
y_2 = PiecewiseAffineFunction([AffineFunction((1/2, -1/2), Polyhedron(vertices=[[-1], [1]]))])
cdp2 = CDP([y_1, y_2], base2)

In [540]:
cdp1.equal(cdp2)

perm  (0, 1)
[[1.]]
[[1.]]
A vertex at (-1) A vertex at (-1)
0.0 1
A vertex at (1) A vertex at (1)
1.0 0
perm  (1, 0)
[[-1.]]
[[-1.]]
A vertex at (1) A vertex at (-1)
0.0 1
A vertex at (-1) A vertex at (1)
1.0 0


False